In [1]:
import scipy.misc as smi
import os
import numpy as np
from glob import glob
from tqdm import tqdm
from PIL import Image
import imageio
import torch
import random


In [3]:
files=glob('/data2/Syria_img/Syria_samples/samples/*/*.png')

In [86]:
for file in tqdm(files):
    image_tensor=torch.tensor(np.array(Image.open(file)).transpose(2,0,1).astype(np.float64))
    # print(image_tensor.shape)
    image_upsample_tensor = torch.nn.functional.interpolate(image_tensor.unsqueeze_(0), size=[6, 6], mode='area')
    img=np.array(image_upsample_tensor[0])
    torch.save(img,file.replace('.png','.pth').replace('Syria_samples/samples','Syria_samples/sample_size6_AREA'))


100%|██████████| 527540/527540 [07:36<00:00, 1154.48it/s]


In [88]:
train=torch.load('/data2/Syria_img/Syria_samples/split_havedamaged/train.pth')
val=torch.load('/data2/Syria_img/Syria_samples/split_havedamaged/val.pth')
print(len(train),len(val))
train_size6=[[i[0].replace('.png','.pth').replace('Syria_samples/samples','Syria_samples/sample_size6_AREA'),i[1]] for i in train]
val_size6=[[i[0].replace('.png','.pth').replace('Syria_samples/samples','Syria_samples/sample_size6_AREA'),i[1]] for i in val]
print(len(train_size6),len(val_size6))
torch.save(train_size6,'/data2/Syria_img/Syria_samples/split_havedamaged_size6_AREA/train.pth')
torch.save(val_size6,'/data2/Syria_img/Syria_samples/split_havedamaged_size6_AREA/val.pth')

175846 87924
175846 87924


In [89]:
for file in tqdm(files):
    img=np.array(Image.open(file).resize((6,6),resample=Image.BOX)).transpose(2,0,1)
    torch.save(img,file.replace('.png','.pth').replace('Syria_samples/samples','Syria_samples/sample_size6_AREA_int'))

100%|██████████| 527540/527540 [06:15<00:00, 1404.82it/s]


In [90]:
train=torch.load('/data2/Syria_img/Syria_samples/split_havedamaged/train.pth')
val=torch.load('/data2/Syria_img/Syria_samples/split_havedamaged/val.pth')
print(len(train),len(val))
train_size6=[[i[0].replace('.png','.pth').replace('Syria_samples/samples','Syria_samples/sample_size6_AREA_int'),i[1]] for i in train]
val_size6=[[i[0].replace('.png','.pth').replace('Syria_samples/samples','Syria_samples/sample_size6_AREA_int'),i[1]] for i in val]
print(len(train_size6),len(val_size6))
torch.save(train_size6,'/data2/Syria_img/Syria_samples/split_havedamaged_size6_AREA_int/train.pth')
torch.save(val_size6,'/data2/Syria_img/Syria_samples/split_havedamaged_size6_AREA_int/val.pth')

175846 87924
175846 87924


In [4]:
train=torch.load('/data2/Syria_img/Syria_samples/split_havedamaged_size6_AREA/train.pth')
val=torch.load('/data2/Syria_img/Syria_samples/split_havedamaged_size6_AREA/val.pth')
cities=list(set([i[0].split('_')[-3].split('/')[-1] for i in train]))
all=train+val

In [12]:
train=torch.load('/data2/Syria_img/Syria_samples/split_havedamaged/train.pth')
val=torch.load('/data2/Syria_img/Syria_samples/split_havedamaged/val.pth')
cities=list(set([i[0].split('_')[-3].split('/')[-1] for i in train]))
all=train+val

In [13]:
samples_cities={}
for city in cities:
    samples_cities[city]=[i for i in all if city in i[0]]
    print(city,len(samples_cities[city]))

Idlib 10232
Deir-ez-Zor 26128
Hama 23788
Aleppo 169692
Raqqa 14626
Homs 19304


In [11]:
for i in cities:
    # torch.save(samples_cities[i],f'/data2/Syria_img/Syria_samples/split_havedamaged_size6_AREA/cities/{i}/sample.pth')
    
    # os.makedirs(f'/data2/Syria_img/Syria_samples/split_havedamaged_size6_AREA/cities/{i}')

In [14]:
samples_A=samples_cities['Hama']
samples_A_n=[i for i in samples_A if i[1]=='n']
samples_A_p=[i for i in samples_A if i[1]=='p']
k_n=int(len(samples_A_n)*7/10)
k_p=int(len(samples_A_p)*7/10)
samples_A_nt=samples_A_n[:k_n]
samples_A_nv=samples_A_n[k_n:]
samples_A_pt=samples_A_p[:k_p]
samples_A_pv=samples_A_p[k_p:]

samples_A_t=samples_A_nt+samples_A_pt
samples_A_v=samples_A_nv+samples_A_pv
random.shuffle(samples_A_t)
random.shuffle(samples_A_v)
print('        n   p   total')
print('train',len(samples_A_nt),len(samples_A_pt),len(samples_A_t))
print('val   ',len(samples_A_nv),len(samples_A_pv),len(samples_A_v))

        n   p   total
train 16017 634 16651
val    6865 272 7137


In [10]:
torch.save(samples_A_v,'/data2/Syria_img/Syria_samples/split_havedamaged_size6_AREA/cities/Hama/val_size120.pth')
torch.save(samples_A_t,'/data2/Syria_img/Syria_samples/split_havedamaged_size6_AREA/cities/Hama/train_size120.pth')

In [10]:
import numpy as np
def get_weight(class_num, pixel_count,mod=1):
    if mod==1:
        W = 1 / np.log(pixel_count)

        #除以np.sum(W)，是将权重归一化，让每个类别的权重相加为1。
        #乘以class_num，是为了让权重中的每个类别的权重值接近1，使网络在一个正常的水平上进行训练。
        W = class_num * W/np.sum(W)  
        
        return W
    else:
        W = 1 / np.array(pixel_count)/2*np.array(pixel_count).sum()
        return W
get_weight(2,[16017,634])

array([0.79983475, 1.20016525])

In [12]:
get_weight(2,[705855,28384],1)

array([0.86452432, 1.13547568])

In [6]:
files=glob('/data2/Syria_img/Syria_samples/samples_enhance/*/*.png')
for file in tqdm(files):
    image_tensor=torch.tensor(np.array(Image.open(file)).transpose(2,0,1).astype(np.float64))
    # print(image_tensor.shape)
    image_upsample_tensor = torch.nn.functional.interpolate(image_tensor.unsqueeze_(0), size=[6, 6], mode='area')
    img=np.array(image_upsample_tensor[0])
    torch.save(img,file.replace('.png','.pth').replace('Syria_samples/samples_enhance','Syria_samples/samples_enhance_size6_AREA'))

100%|██████████| 247184/247184 [03:32<00:00, 1163.75it/s]


In [13]:
train=torch.load('/data2/Syria_img/Syria_samples/split_havedamaged_enhance_Hama/train.pth')
val=torch.load('/data2/Syria_img/Syria_samples/split_havedamaged_enhance_Hama/val.pth')
print(len(train),len(val))
train_size6=[[i[0].replace('.png','.pth').replace('Syria_samples/samples_enhance','Syria_samples/samples_enhance_size6_AREA'),i[1]] for i in train]
val_size6=[[i[0].replace('.png','.pth').replace('Syria_samples/samples_enhance','Syria_samples/samples_enhance_size6_AREA'),i[1]] for i in val]
print(len(train_size6),len(val_size6))
torch.save(train_size6,'/data2/Syria_img/Syria_samples/split_havedamaged_enhance_Hama_size6_AREA/train.pth')
torch.save(val_size6,'/data2/Syria_img/Syria_samples/split_havedamaged_enhance_Hama_size6_AREA/val.pth')

82394 41198
82394 41198


In [11]:
train_enhance=torch.load('/data2/Syria_img/Syria_samples/split_havedamaged_enhance_Hama/train.pth')
val_enhance=torch.load('/data2/Syria_img/Syria_samples/split_havedamaged_enhance_Hama/val.pth')

In [13]:
train=torch.load('/data2/Syria_img/Syria_samples/split_havedamaged_size6_AREA/cities/Hama/train_size120.pth')
val=torch.load('/data2/Syria_img/Syria_samples/split_havedamaged_size6_AREA/cities/Hama/val_size120.pth')

In [14]:
val[0]

['/data2/Syria_img/Syria_samples/samples/pre/Hama_C_3065.png', 'n']

In [15]:
train_all=train+train_enhance
val_all=val+val_enhance
random.shuffle(train_all)
random.shuffle(val_all)

In [16]:
torch.save(train_all,'/data2/Syria_img/Syria_samples/split_havedamaged_all_Hama/train.pth')
torch.save(val_all,'/data2/Syria_img/Syria_samples/split_havedamaged_all_Hama/val.pth')

In [19]:
train=torch.load('/data2/Syria_img/Syria_samples/split_havedamaged_all_Hama_size6_AREA/train.pth')
val=torch.load('/data2/Syria_img/Syria_samples/split_havedamaged_all_Hama_size6_AREA/val.pth')

In [20]:
keys={'label':['B','C'],
        'before':['enhance-A-B'],
        'after':['enhance-D-Z'],
        'interval':['enhance-B-C','enhance-C-D']}
split_split={}
for k in keys.keys():
        s=[]
        for kk in keys[k]:
                for i in val:
                        id=os.path.basename(i[0]).split('_')[1]
                        if len(id.split('-'))>1:
                                id=id[:-2]
                        if id==kk:
                                s.append(i)
        split_split[k]=s


In [21]:
split_split.keys()

dict_keys(['label', 'before', 'after', 'interval'])

In [23]:
for k in split_split.keys():
    print(k,len(split_split[k]))
    torch.save(split_split[k],f'/data2/Syria_img/Syria_samples/split_havedamaged_all_Hama_size6_AREA/val_{k}.pth')

label 7137
before 12023
after 1535
interval 27640


In [14]:

cities=['Homs', 'Idlib', 'Raqqa', 'Aleppo', 'Deir-ez-Zor', 'Hama']

In [3]:
for city in cities:
    print(city)
    all_info=torch.load(f'/ssd/hk/Syria_samples/split_havedamaged_size120_enhance/cities/{city}/train.pth')+torch.load(f'/ssd/hk/Syria_samples/split_havedamaged_size120_enhance/cities/{city}/val.pth')
    files=[i[0] for i in all_info]
    files_post=[i.replace('pre','post') for i in files]
    print('interpolate files')
    time.sleep(2)
    for file in tqdm(files_post):
        image_tensor=torch.tensor(np.array(Image.open(file)).transpose(2,0,1).astype(np.float64))
        # print(image_tensor.shape)
        image_upsample_tensor = torch.nn.functional.interpolate(image_tensor.unsqueeze_(0), size=[6, 6], mode='area')
        img=np.array(image_upsample_tensor[0])
        torch.save(img,file.replace('.png','.pth').replace('Syria_samples/samples_enhance','Syria_samples/samples_enhance_size6_AREA'))
    print('interpolate post files')
    time.sleep(2)
    for file in tqdm(files):
        image_tensor=torch.tensor(np.array(Image.open(file)).transpose(2,0,1).astype(np.float64))
        # print(image_tensor.shape)
        image_upsample_tensor = torch.nn.functional.interpolate(image_tensor.unsqueeze_(0), size=[6, 6], mode='area')
        img=np.array(image_upsample_tensor[0])
        torch.save(img,file.replace('.png','.pth').replace('Syria_samples/samples_enhance','Syria_samples/samples_enhance_size6_AREA'))


Raqqa
interpolate files


100%|██████████| 51678/51678 [00:42<00:00, 1215.61it/s]


interpolate post files


100%|██████████| 51678/51678 [00:44<00:00, 1164.93it/s]


Homs
interpolate files


...

100%|██████████| 508662/508662 [08:34<00:00, 988.27it/s] 


Deir-ez-Zor
interpolate files


100%|██████████| 46148/46148 [00:46<00:00, 994.74it/s] 


interpolate post files


100%|██████████| 46148/46148 [00:47<00:00, 981.71it/s] 


In [5]:
file.replace('.png','.pth').replace('Syria_samples/samples_enhance','Syria_samples/samples_enhance_size6_AREA')

'/ssd/hk/Syria_samples/samples_enhance_size6_AREA/pre/Raqqa_enhance-A-B-4_3247.pth'

In [4]:
for city in cities:
    print(city)
    train=torch.load(f'/ssd/hk/Syria_samples/split_havedamaged_size120_enhance/cities/{city}/train.pth')
    val=torch.load(f'/ssd/hk/Syria_samples/split_havedamaged_size120_enhance/cities/{city}/val.pth')
    print(len(train),len(val))
    train_size6=[[i[0].replace('.png','.pth').replace('Syria_samples/samples_enhance','Syria_samples/samples_enhance_size6_AREA'),i[1]] for i in train]
    val_size6=[[i[0].replace('.png','.pth').replace('Syria_samples/samples_enhance','Syria_samples/samples_enhance_size6_AREA'),i[1]] for i in val]
    print(len(train_size6),len(val_size6))
    torch.save(train_size6,f'/ssd/hk/Syria_samples/split_havedamaged_size6_enhance/cities/{city}/train.pth')
    torch.save(val_size6,f'/ssd/hk/Syria_samples/split_havedamaged_size6_enhance/cities/{city}/val.pth')

Raqqa
34451 17227
34451 17227
Homs
32203 16103
32203 16103
Idlib
13668 6835
13668 6835
Aleppo
339108 169554
339108 169554
Deir-ez-Zor
30765 15383
30765 15383


In [ ]:
    print(city)
    train=torch.load(f'/ssd/hk/Syria_samples/split_havedamaged_size120_enhance/cities/{city}/train.pth')
    val=torch.load(f'/ssd/hk/Syria_samples/split_havedamaged_size120_enhance/cities/{city}/val.pth')
    print(len(train),len(val))
    train_size6=[[i[0].replace('.png','.pth').replace('Syria_samples/samples_enhance','Syria_samples/samples_enhance_size6_AREA'),i[1]] for i in train]
    val_size6=[[i[0].replace('.png','.pth').replace('Syria_samples/samples_enhance','Syria_samples/samples_enhance_size6_AREA'),i[1]] for i in val]
    print(len(train_size6),len(val_size6))
    torch.save(train_size6,f'/ssd/hk/Syria_samples/split_havedamaged_size6_enhance/cities/{city}/train.pth')
    torch.save(val_size6,f'/ssd/hk/Syria_samples/split_havedamaged_size6_enhance/cities/{city}/val.pth')

In [16]:
for city in cities:
    print(city)
    train=torch.load(f'/ssd/hk/Syria_samples/split_havedamaged_size6_enhance/cities/{city}/train.pth')+torch.load(f'/ssd/hk/Syria_samples/split_havedamaged_size6/cities/{city}/train.pth')
    val=torch.load(f'/ssd/hk/Syria_samples/split_havedamaged_size6_enhance/cities/{city}/val.pth')+torch.load(f'/ssd/hk/Syria_samples/split_havedamaged_size6/cities/{city}/val.pth')
    random.shuffle(train)
    random.shuffle(val)
    torch.save(train,f'/ssd/hk/Syria_samples/split_havedamaged_size6_all/cities/{city}/train.pth')
    torch.save(val,f'/ssd/hk/Syria_samples/split_havedamaged_size6_all/cities/{city}/val.pth')

Homs
Idlib
Raqqa
Aleppo
Deir-ez-Zor
Hama


In [18]:
for city in cities:
    print(city)
    train=torch.load(f'/ssd/hk/Syria_samples/split_havedamaged_size6_all/cities/{city}/train.pth')
    val=torch.load(f'/ssd/hk/Syria_samples/split_havedamaged_size6_all/cities/{city}/val.pth')
    for i in train:
        if not os.path.exists(i[0]):
            print(i[0])
    for i in val:
        if not os.path.exists(i[0]):
            print(i[0])
    print(len(train)+len(val))

Homs
67610
Idlib
30735
Raqqa
66304
Aleppo
678354
Deir-ez-Zor
72276
Hama
147380


In [35]:
train=[]
val=[]
for city in cities:
    train+=torch.load(f'/ssd/hk/Syria_samples/split_havedamaged_size6_enhance/cities/{city}/train.pth')
    val+=torch.load(f'/ssd/hk/Syria_samples/split_havedamaged_size6_enhance/cities/{city}/val.pth')
random.shuffle(train)
random.shuffle(val)
print(len(train)+len(val))
torch.save(train,'/ssd/hk/Syria_samples/split_havedamaged_size6_enhance/train.pth')
torch.save(val,'/ssd/hk/Syria_samples/split_havedamaged_size6_enhance/val.pth')

798889


In [ ]:
for i in train:
    if not os.path.exists(i[0]):
        print(i)

In [ ]:
for i in val:
    if not os.path.exists(i[0]):
        print(i)

In [15]:
from glob import glob
print(len(glob('/data2/Syria_img/Syria_samples/samples/post/*.png')))
print(len(glob('/ssd/hk/Syria_samples/samples/post/*.png')))

263770
263770


'/data2/Syria_img/Syria_samples/samples/pre/Aleppo_E_24802.png'

In [13]:
import gdal
import numpy as np
import pandas as pd
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import torch
from glob import glob
import shapely
import random
from tqdm import tqdm
from collections import Counter
from pybob.GeoImg import GeoImg
import osr
from pybob.image_tools import create_mask_from_shapefile
import gc
import imageio
import time
import shutil
cities=['Homs', 'Idlib', 'Raqqa', 'Aleppo', 'Deir-ez-Zor', 'Hama']
def match(string,input_list):
    new_data_list=[i for i in input_list if  re.match(string, i[0]) != None]
    return new_data_list
city_sample=[]
for city in cities:
    t=torch.load(f'/ssd/hk/Syria_samples/coords_split/{city}_train.pth')
    v=torch.load(f'/ssd/hk/Syria_samples/coords_split/{city}_val.pth')
    print(city)
    city_sample=[]
    for i in glob(f'/ssd/hk/Syria_samples/samples_v2/{city}*v2.pth'):
        city_sample+=torch.load(i)['info']
        
    for i in glob(f'/ssd/hk/Syria_samples/samples_enhance_v2/{city}*v2.pth'):
        city_sample+=torch.load(i)['info']


    for i in range(len(city_sample)):
        if len(city_sample[i][3])==2:
            if city_sample[i][3][0] is not None:
                city_sample[i][3]=city_sample[i][3][0]
            else:
                city_sample[i][3]=city_sample[i][3][1]
    city_pathes=[[i[0],'n'] if i[3][0]==0 else [i[0],'p'] for i in city_sample]
    random.shuffle(city_pathes)
    import re
    city_coord_sample={}
    city_coord_sample['train']={}
    city_coord_sample['val']={}
    print('match train')
    time.sleep(2)
    for i in t:
        city_coord_sample['train'][i]=[]
    for i in v:
        city_coord_sample['val'][i]=[]

    for i in tqdm(city_pathes):
        id=i[0].split('_')[-1].split('.')[0]
        if id in t:
            city_coord_sample['train'][id].append(i)
        if id in v:
            city_coord_sample['val'][id].append(i)
    for i in t:
        city_coord_sample['train'][i].sort()
    for i in v:
        city_coord_sample['val'][i].sort()
    torch.save(city_coord_sample,f'/ssd/hk/Syria_samples/split_havedamaged_size120_all_v2/{city}.pth') 

Homs
match train


100%|██████████| 67610/67610 [00:06<00:00, 11191.42it/s]


Idlib
match train


100%|██████████| 30735/30735 [00:00<00:00, 40138.89it/s]


Raqqa
match train


...

100%|██████████| 678354/678354 [04:45<00:00, 2378.47it/s]


Deir-ez-Zor
match train


100%|██████████| 72276/72276 [00:05<00:00, 13811.27it/s]


Hama
match train


100%|██████████| 147380/147380 [00:19<00:00, 7636.90it/s]


In [15]:
cities=['Homs', 'Idlib', 'Raqqa', 'Aleppo', 'Deir-ez-Zor', 'Hama']
train=[]
val=[]
for city in cities:
    print(city)
    time.sleep(2)
    samples=torch.load(f'/ssd/hk/Syria_samples/split_havedamaged_size120_all_v2/{city}.pth')
    ts=samples['train']
    vs=samples['val']
    for c in tqdm(ts.keys()):
        train+=ts[c]
    for c in tqdm(vs.keys()):
        val+=vs[c]   

Homs


100%|██████████| 2896/2896 [00:00<00:00, 1893779.92it/s]


Idlib


100%|██████████| 768/768 [00:00<00:00, 1750666.02it/s]


Raqqa


...

100%|██████████| 12586/12586 [00:00<00:00, 2016021.01it/s]


Deir-ez-Zor


100%|██████████| 1961/1961 [00:00<00:00, 1654269.94it/s]


Hama


100%|██████████| 3569/3569 [00:00<00:00, 2717768.88it/s]


In [16]:
random.shuffle(train)
random.shuffle(val)

In [17]:
print(len(train))
print(len(val))

734239
314794


In [20]:
torch.save(train,'/ssd/hk/Syria_samples/split_havedamaged_size120_all_v2/train.pth')
torch.save(val,'/ssd/hk/Syria_samples/split_havedamaged_size120_all_v2/val.pth')

In [4]:
import torch
print(len(torch.load('/ssd/hk/Syria_samples/split_havedamaged_size120_all_v2/train.pth')))
print(len(torch.load('/ssd/hk/Syria_samples/split_havedamaged_size120_all_v2/val.pth')))
print(len(torch.load('/ssd/hk/Syria_samples/split_havedamaged_size120_all/train.pth')))
print(len(torch.load('/ssd/hk/Syria_samples/split_havedamaged_size120_all/val.pth')))

963018
413464
717222
345437


In [5]:
len(os.listdir('/ssd/hk/Syria_samples/samples/pre'))

263770

In [6]:
len(os.listdir('/ssd/hk/Syria_samples/samples_enhance/pre'))

798889

In [7]:
263770+798889

1062659

In [8]:
717222+345437

1062659

In [18]:
734239+314794

1049033

In [11]:
cities=['Homs', 'Idlib', 'Raqqa', 'Aleppo', 'Deir-ez-Zor', 'Hama']
def match(string,input_list):
    new_data_list=[i for i in input_list if  re.match(string, i[0]) != None]
    return new_data_list
city_sample=[]
a=0
for city in cities:
    t=torch.load(f'/ssd/hk/Syria_samples/coords_split/{city}_train.pth')
    v=torch.load(f'/ssd/hk/Syria_samples/coords_split/{city}_val.pth')
    print(city)
    for i in glob(f'/ssd/hk/Syria_samples/samples_v2/{city}*v2.pth'):
        city_sample+=torch.load(i)['info']
    
    for i in glob(f'/ssd/hk/Syria_samples/samples_enhance_v2/{city}*v2.pth'):
        city_sample+=torch.load(i)['info']
    print(len(city_sample))
    a+=len(city_sample)
print(a)

Homs
67610
Idlib
98345
Raqqa
164649
Aleppo
843003
Deir-ez-Zor
915279
Hama
1062659
3151545


In [23]:
for city in cities:
    t=torch.load(f'/ssd/hk/Syria_samples/coords_split/{city}_train.pth')
    v=torch.load(f'/ssd/hk/Syria_samples/coords_split/{city}_val.pth')
    for i in t:
        if i in v:
            print(i)
    for i in v:
        if i in t:
            print(i)

In [29]:
print(len([i[1] for i in train if i[1]=='p']))
print(len([i[1] for i in train if i[1]=='n']))
print(len(train))

28384
705855
734239


In [30]:
train=torch.load('/ssd/hk/Syria_samples/split_havedamaged_size120_all/train.pth')
print(len([i[1] for i in train if i[1]=='p']))
print(len([i[1] for i in train if i[1]=='n']))
print(len(train))

27638
689584
717222


In [34]:
val=torch.load('/ssd/hk/Syria_samples/split_havedamaged_size120_all/val.pth')
print(len([i[1] for i in val if i[1]=='p']))
print(len([i[1] for i in val if i[1]=='n']))
print(len(val))

13353
332084
345437


In [35]:
val=torch.load('/ssd/hk/Syria_samples/split_havedamaged_size120_all_v2/val.pth')
print(len([i[1] for i in val if i[1]=='p']))
print(len([i[1] for i in val if i[1]=='n']))
print(len(val))

12364
302430
314794


In [2]:
import torch
val=torch.load('/ssd/hk/Syria_samples/split_havedamaged_size120_all_v2/val.pth')
train=torch.load('/ssd/hk/Syria_samples/split_havedamaged_size120_all_v2/train.pth')
print(len(val),len(train))

314794 734239


In [44]:
val[1][0].replace('/samples_v2/','/samples_size6_AREA_v2/').replace('/samples_enhance_v2/','/samples_enhance_size6_AREA_v2/')

'/ssd/hk/Syria_samples/samples_enhance_v2/pre/Aleppo_enhance-C-D-3_114138.png'

In [47]:

    all_info=torch.load('/ssd/hk/Syria_samples/split_havedamaged_size120_all_v2/val.pth')+torch.load('/ssd/hk/Syria_samples/split_havedamaged_size120_all_v2/train.pth')
    files=[i[0] for i in all_info]
    files_post=[i.replace('pre','post') for i in files]
    print('interpolate files')  
    time.sleep(2)
    for file in tqdm(files_post):
        image_tensor=torch.tensor(np.array(Image.open(file)).transpose(2,0,1).astype(np.float64))
        # print(image_tensor.shape)
        image_upsample_tensor = torch.nn.functional.interpolate(image_tensor.unsqueeze_(0), size=[6, 6], mode='area')
        img=np.array(image_upsample_tensor[0])
        torch.save(img,file.replace('.png','.pth').replace('/samples_v2/','/samples_size6_AREA_v2/').replace('/samples_enhance_v2/','/samples_enhance_size6_AREA_v2/'))
    print('interpolate post files')
    time.sleep(2)
    for file in tqdm(files):
        image_tensor=torch.tensor(np.array(Image.open(file)).transpose(2,0,1).astype(np.float64))
        # print(image_tensor.shape)
        image_upsample_tensor = torch.nn.functional.interpolate(image_tensor.unsqueeze_(0), size=[6, 6], mode='area')
        img=np.array(image_upsample_tensor[0])
        torch.save(img,file.replace('.png','.pth').replace('/samples_v2/','/samples_size6_AREA_v2/').replace('/samples_enhance_v2/','/samples_enhance_size6_AREA_v2/'))


interpolate files


100%|██████████| 1049033/1049033 [14:34<00:00, 1200.20it/s]


interpolate post files


100%|██████████| 1049033/1049033 [14:53<00:00, 1174.00it/s]


In [ ]:
i

In [22]:
t=[]
v=[]
for i in tqdm(train):
    ii=[i[0].replace('.png','.pth').replace('/samples_v2/','/samples_size6_AREA_v2/').replace('/samples_enhance_v2/','/samples_enhance_size6_AREA_v2/'),i[1]]
    if not os.path.exists(ii[0]):
        print(ii)
    t.append(ii)
for i in tqdm(val):
    ii=[i[0].replace('.png','.pth').replace('/samples_v2/','/samples_size6_AREA_v2/').replace('/samples_enhance_v2/','/samples_enhance_size6_AREA_v2/'),i[1]]
    if not os.path.exists(ii[0]):
        print(ii)
    v.append(ii)

100%|██████████| 314794/314794 [00:01<00:00, 238510.60it/s]


In [9]:
torch.save(t,'/ssd/hk/Syria_samples/split_havedamaged_size6_all_v2/train.pth')
torch.save(v,'/ssd/hk/Syria_samples/split_havedamaged_size6_all_v2/val.pth')

In [18]:
len([i[1] for i in t if i[1]=='p'])

28384

In [19]:
len([i[1] for i in torch.load('/ssd/hk/Syria_samples/split_havedamaged_size6_all_v2/train.pth') if i[1]=='p'])

28384

In [32]:
from glob import glob
import os
from tqdm import tqdm
import time
cities=os.listdir('/ssd/hk/Syria_samples/split_havedamaged_size120_all_v3/cities/')

In [27]:
cities

['Idlib', 'Raqqa', 'Homs', 'Aleppo', 'Hama', 'HamaandRaqqa', 'Deir-ez-Zor']

In [12]:
import torch
val=torch.load('/ssd/hk/Syria_samples/split_havedamaged_size120_all_v3/cities/Hama/val.pth')+torch.load('/ssd/hk/Syria_samples/split_havedamaged_size120_all_v3/cities/Raqqa/val.pth')

In [13]:
torch.save(val,'/ssd/hk/Syria_samples/split_havedamaged_size120_all_v3/cities/HamaandRaqqa/val.pth')

In [30]:
def get_data6(data):
    data6=[]
    for s in tqdm(data):
        s6=[]
        p6=[]
        for i in s[0]:
            if i is not None:
                i=i.replace('png','pth')
                i=i.replace('/samples_v2/','/samples_size6_AREA_v2/')
                i=i.replace('/samples_enhance_v2/','/samples_enhance_size6_AREA_v2/')
                if not os.path.exists(i):
                    print(i)
                else:
                    p6.append(i)
            else:
                p6.append(None)
        s6.append(p6)
        s6.append(s[1])
        data6.append(s6)
    return data6


In [33]:

fs=glob('/ssd/hk/Syria_samples/split_havedamaged_size120_all_v3/cities/*/*')
for f in fs:
    print(f)
    time.sleep(2)
    data=torch.load(f)
    data6=get_data6(data)
    torch.save(data6,f.replace('size120','size6'))

/ssd/hk/Syria_samples/split_havedamaged_size120_all_v3/cities/Idlib/val.pth


100%|██████████| 768/768 [00:00<00:00, 23573.16it/s]


/ssd/hk/Syria_samples/split_havedamaged_size120_all_v3/cities/Idlib/train.pth


100%|██████████| 1790/1790 [00:00<00:00, 28723.71it/s]


/ssd/hk/Syria_samples/split_havedamaged_size120_all_v3/cities/Raqqa/val.pth


...

100%|██████████| 13440/13440 [00:00<00:00, 31391.75it/s]


/ssd/hk/Syria_samples/split_havedamaged_size120_all_v3/cities/Deir-ez-Zor/val.pth


100%|██████████| 1961/1961 [00:00<00:00, 30275.70it/s]


/ssd/hk/Syria_samples/split_havedamaged_size120_all_v3/cities/Deir-ez-Zor/train.pth


100%|██████████| 4571/4571 [00:00<00:00, 28588.80it/s]


In [35]:
import numpy as np

In [36]:
fs=glob('/ssd/hk/Syria_samples/split_havedamaged_size120_all_v3/cities/*/*')
for f in fs:
    print(f)
    time.sleep(2)
    data=torch.load(f)
    l=[]
    for d in data:
        l.append(len([i for i in d[0] if i is not None]))
    print(np.array(l).max())

/ssd/hk/Syria_samples/split_havedamaged_size120_all_v3/cities/Idlib/val.pth
15
/ssd/hk/Syria_samples/split_havedamaged_size120_all_v3/cities/Idlib/train.pth
15
/ssd/hk/Syria_samples/split_havedamaged_size120_all_v3/cities/Raqqa/val.pth
14
/ssd/hk/Syria_samples/split_havedamaged_size120_all_v3/cities/Raqqa/train.pth
14
/ssd/hk/Syria_samples/split_havedamaged_size120_all_v3/cities/Homs/val.pth
9
/ssd/hk/Syria_samples/split_havedamaged_size120_all_v3/cities/Homs/train.pth
9
/ssd/hk/Syria_samples/split_havedamaged_size120_all_v3/cities/Aleppo/val.pth
17
/ssd/hk/Syria_samples/split_havedamaged_size120_all_v3/cities/Aleppo/train.pth
17
/ssd/hk/Syria_samples/split_havedamaged_size120_all_v3/cities/Hama/val.pth
16
/ssd/hk/Syria_samples/split_havedamaged_size120_all_v3/cities/Hama/train.pth
16
/ssd/hk/Syria_samples/split_havedamaged_size120_all_v3/cities/HamaandRaqqa/val.pth
16
/ssd/hk/Syria_samples/split_havedamaged_size120_all_v3/cities/HamaandRaqqa/train.pth
16
/ssd/hk/Syria_samples/split_ha

In [42]:
fs=glob('/ssd/hk/Syria_samples/split_havedamaged_size6_all_v3/cities/*/*')
for f in fs:
    print(f)
    time.sleep(2)
    data=torch.load(f)
    data2=[[i[0][:17],i[1][:17]] for i in data]
    torch.save(data2,f)

/ssd/hk/Syria_samples/split_havedamaged_size6_all_v3/cities/Idlib/val.pth
/ssd/hk/Syria_samples/split_havedamaged_size6_all_v3/cities/Idlib/train.pth
/ssd/hk/Syria_samples/split_havedamaged_size6_all_v3/cities/Raqqa/val.pth
/ssd/hk/Syria_samples/split_havedamaged_size6_all_v3/cities/Raqqa/train.pth
/ssd/hk/Syria_samples/split_havedamaged_size6_all_v3/cities/Homs/val.pth
/ssd/hk/Syria_samples/split_havedamaged_size6_all_v3/cities/Homs/train.pth
/ssd/hk/Syria_samples/split_havedamaged_size6_all_v3/cities/Aleppo/val.pth
/ssd/hk/Syria_samples/split_havedamaged_size6_all_v3/cities/Aleppo/train.pth
/ssd/hk/Syria_samples/split_havedamaged_size6_all_v3/cities/Hama/val.pth
/ssd/hk/Syria_samples/split_havedamaged_size6_all_v3/cities/Hama/train.pth
/ssd/hk/Syria_samples/split_havedamaged_size6_all_v3/cities/HamaandRaqqa/val.pth
/ssd/hk/Syria_samples/split_havedamaged_size6_all_v3/cities/HamaandRaqqa/train.pth
/ssd/hk/Syria_samples/split_havedamaged_size6_all_v3/cities/Deir-ez-Zor/val.pth
/ssd/hk/

In [47]:
sample_list[0][0]

['/ssd/hk/Syria_samples/samples_enhance_size6_AREA_v2/pre/Deir-ez-Zor_enhance-A-B-1_010049.pth',
 '/ssd/hk/Syria_samples/samples_enhance_size6_AREA_v2/post/Deir-ez-Zor_enhance-A-B-1_010049.pth',
 '/ssd/hk/Syria_samples/samples_enhance_size6_AREA_v2/post/Deir-ez-Zor_enhance-A-B-2_010049.pth',
 '/ssd/hk/Syria_samples/samples_enhance_size6_AREA_v2/post/Deir-ez-Zor_enhance-A-B-3_010049.pth',
 '/ssd/hk/Syria_samples/samples_size6_AREA_v2/post/Deir-ez-Zor_B_010049.pth',
 '/ssd/hk/Syria_samples/samples_size6_AREA_v2/post/Deir-ez-Zor_C_010049.pth',
 '/ssd/hk/Syria_samples/samples_enhance_size6_AREA_v2/post/Deir-ez-Zor_enhance-C-D-1_010049.pth',
 '/ssd/hk/Syria_samples/samples_enhance_size6_AREA_v2/post/Deir-ez-Zor_enhance-C-D-2_010049.pth',
 '/ssd/hk/Syria_samples/samples_enhance_size6_AREA_v2/post/Deir-ez-Zor_enhance-C-D-3_010049.pth',
 '/ssd/hk/Syria_samples/samples_size6_AREA_v2/post/Deir-ez-Zor_D_010049.pth',
 '/ssd/hk/Syria_samples/samples_enhance_size6_AREA_v2/post/Deir-ez-Zor_enhance-D-

In [45]:
sample_list=data2

In [56]:

path_pre=sample_list[0][0][0]
path_post=sample_list[0][0][1:]
img_pre=torch.load(path_pre)
imgs=[np.concatenate([img_pre,torch.load(post)],axis=0) if post is not None else np.zeros((6,6,6)) for post in path_post]

In [59]:
img=np.concatenate(imgs,axis=0)

In [60]:
img.shape

(96, 6, 6)

In [75]:
import torch
a=torch.load('/ssd/hk/Syria_samples/split_havedamaged_size6_all_v3/cities/HamaandRaqqa/val.pth')
a[0][0][3]

'/ssd/hk/Syria_samples/samples_enhance_size6_AREA_v2/post/Hama_enhance-A-B-3_089052.pth'

In [66]:
b=torch.load('/ssd/hk/Syria_samples/split_havedamaged_size6_all_v3/cities/HamaandRaqqa/train.pth')

In [69]:
z=[]
for i in range(len(b)):
    z+=b[i][0]
len(z)

228480

In [74]:
for x in range(100):
    xx=random.randint(0,len(a))
    for y in range(100):
        yy=random.randint(0,15)
        if a[xx][0][yy] is not None:
            if a[xx][0][yy] in z:
                print(a[xx][0][yy])


In [72]:
import random
random.randint(0,100)

25

In [98]:
index=0
path_pre=a[index][0][0]
path_post_=a[index][0][1:]
labels_=a[index][1][1:]

path_post=[]
labels=[]
mask_num=random.randint(0,5)
mask=random.sample(range(1, 14), mask_num)
print(len(labels_))
print(len(mask))
for i in range(len(path_post_)):
    if i not in mask:
        path_post.append(path_post_[i])
        labels.append(labels_[i])
print(len(labels))
path_post+=[None]*len(mask)
labels+=[None]*len(mask)
print(len(labels))
# print(path_post)

16
0
16
16


In [3]:
import torch 
import random
t=torch.load('/ssd/hk/Syria_samples/split_havedamaged_size6_all_v3/cities/HamaandRaqqa/train.pth')
v=torch.load('/ssd/hk/Syria_samples/split_havedamaged_size6_all_v3/cities/HamaandRaqqa/val.pth')
all=t+v
random.shuffle(all)
split=int(len(all)*0.7)
tt=all[:split]
vv=all[split:]
torch.save(tt,'/ssd/hk/Syria_samples/split_havedamaged_size6_all_v3/cities/HamaandRaqqa/train.pth')
torch.save(vv,'/ssd/hk/Syria_samples/split_havedamaged_size6_all_v3/cities/HamaandRaqqa/val.pth')

In [2]:
vv

NameError: name 'vv' is not defined

In [14]:
import numpy as np
for i in range(100000):
    k=random.randint(1,16)
    labels_=[1]*k+[0]*(16-k)
    # print(np.array(labels_).sum())
    l=len([i for i in labels_ if i!=0])
    # if self.is_train:
    # path_post=[]
    labels=[]
    mask_num=random.randint(0,l-1)
    mask=random.sample(range(0, l), mask_num)
    # print(len(labels_))
    # print(len(mask))
    for i in range(len(labels_)):
        if i not in mask:
            # path_post.append(path_post_[i])
            labels.append(labels_[i])
    # print(len(labels))
    # path_post+=[None]*len(mask)
    labels+=[0]*len(mask)
    # print(np.array(labels).sum())
    if np.array(labels).sum()==0:
        print('aaa')
        break

In [9]:
import torch
torch.load('/ssd/hk/Syria_samples/split_havedamaged_size6_all_v3/cities/Alppo/val.pth')

16

In [18]:
import os 
os.listdir('/ssd/hk/Syria_samples/split_havedamaged_size6_all_v3/cities/')

['Idlib', 'Raqqa', 'Homs', 'Aleppo', 'Hama', 'HamaandRaqqa', 'Deir-ez-Zor']

In [21]:
cities=['Idlib', 'Raqqa', 'Homs', 'Aleppo', 'Hama', 'Deir-ez-Zor']
v=[]
t=[]
for city in cities:
    v+=torch.load(f'/ssd/hk/Syria_samples/split_havedamaged_size6_all_v3/cities/{city}/val.pth')
    t+=torch.load(f'/ssd/hk/Syria_samples/split_havedamaged_size6_all_v3/cities/{city}/train.pth')
all=v+t
random.shuffle(v)
random.shuffle(t)
random.shuffle(all)
torch.save(all,f'/ssd/hk/Syria_samples/split_havedamaged_size6_all_v3/samples.pth')
torch.save(t,f'/ssd/hk/Syria_samples/split_havedamaged_size6_all_v3/train.pth')
torch.save(v,f'/ssd/hk/Syria_samples/split_havedamaged_size6_all_v3/val.pth')
print(len(t),len(v),len(all))

55922 23973 79895


In [19]:
import random
for city in cities:
    v=torch.load(f'/ssd/hk/Syria_samples/split_havedamaged_size6_all_v3/cities/{city}/val.pth')
    t=torch.load(f'/ssd/hk/Syria_samples/split_havedamaged_size6_all_v3/cities/{city}/train.pth')
    all=v+t
    random.shuffle(all)
    torch.save(all,f'/ssd/hk/Syria_samples/split_havedamaged_size6_all_v3/cities/{city}/samples.pth')

In [20]:
import random
for city in cities:
    v=torch.load(f'/ssd/hk/Syria_samples/split_havedamaged_size6_all_v2/cities/{city}/val.pth')
    t=torch.load(f'/ssd/hk/Syria_samples/split_havedamaged_size6_all_v2/cities/{city}/train.pth')
    all=v+t
    random.shuffle(all)
    torch.save(all,f'/ssd/hk/Syria_samples/split_havedamaged_size6_all_v2/cities/{city}/samples.pth')

In [12]:
from glob import glob
import torch
from tqdm import tqdm
import numpy as np
from PIL import Image
import shutil

In [23]:
files=glob('/ssd/hk/Syria_samples/samples_addpre/samples/*')


In [24]:
files[0]

'/ssd/hk/Syria_samples/samples_addpre/samples/Homs_A-4_107073.png'

In [25]:
files[0].replace('/samples_addpre/samples/','/samples_addpre_size6_AREA/samples/').replace('.png','.pth')

'/ssd/hk/Syria_samples/samples_addpre_size6_AREA/sampels/Homs_A-4_107073.pth'

In [27]:
for file in tqdm(files):
    image_tensor=torch.tensor(np.array(Image.open(file)).transpose(2,0,1).astype(np.float64))
    # print(image_tensor.shape)
    image_upsample_tensor = torch.nn.functional.interpolate(image_tensor.unsqueeze_(0), size=[6, 6], mode='area')
    img=np.array(image_upsample_tensor[0])
    torch.save(img,file.replace('/samples_addpre/samples/','/samples_addpre_size6_AREA/samples/').replace('.png','.pth'))

100%|██████████| 28956/28956 [00:25<00:00, 1140.15it/s]
